Scripts for regression experiments on mouse

In [1]:
import pandas as pd
from pathlib import Path
import mp_run

from scipy import stats


%load_ext autoreload
%autoreload 2

# regex for number extraction from string
number_pattern =  r'(-?(?:0|[1-9]\d*)(?:\.\d+)?(?:[eE][+-]?\d+)?)'


In [2]:
train_source_df_1 = pd.read_csv('../data/yeast/GSE145936/normalized/train_source.csv.gz', compression='gzip', index_col=0)
train_target_df_1 = pd.read_csv('../data/yeast/GSE145936/normalized/train_target.csv.gz', compression='gzip', index_col=0)
test_source_df_1 = pd.read_csv('../data/yeast/GSE145936/normalized/test_source.csv.gz', compression='gzip', index_col=0)
test_target_df_1 = pd.read_csv('../data/yeast/GSE145936/normalized/test_target.csv.gz', compression='gzip', index_col=0)

train_source_df_2 = pd.read_csv('../data/yeast/GSE153609/normalized/train_source.csv.gz', compression='gzip', index_col=0)
train_target_df_2 = pd.read_csv('../data/yeast/GSE153609/normalized/train_target.csv.gz', compression='gzip', index_col=0)
test_source_df_2 = pd.read_csv('../data/yeast/GSE153609/normalized/test_source.csv.gz', compression='gzip', index_col=0)
test_target_df_2 = pd.read_csv('../data/yeast/GSE153609/normalized/test_target.csv.gz', compression='gzip', index_col=0)

train_source_df_3 = pd.read_csv('../data/yeast/GSE168699/normalized/train_source.csv.gz', compression='gzip', index_col=0)
train_target_df_3 = pd.read_csv('../data/yeast/GSE168699/normalized/train_target.csv.gz', compression='gzip', index_col=0)
test_source_df_3 = pd.read_csv('../data/yeast/GSE168699/normalized/test_source.csv.gz', compression='gzip', index_col=0)
test_target_df_3 = pd.read_csv('../data/yeast/GSE168699/normalized/test_target.csv.gz', compression='gzip', index_col=0)

In [3]:
train_source_4_1 = pd.read_csv('../data/yeast/GSE226769/normalized/train_source_1.csv.gz', compression='gzip', index_col=0)
train_target_4_1 = pd.read_csv('../data/yeast/GSE226769/normalized/train_target_1.csv.gz', compression='gzip', index_col=0)
test_source_4_1 = pd.read_csv('../data/yeast/GSE226769/normalized/test_source_1.csv.gz', compression='gzip', index_col=0)
test_target_4_1 = pd.read_csv('../data/yeast/GSE226769/normalized/test_target_1.csv.gz', compression='gzip', index_col=0)

train_source_4_2 = pd.read_csv('../data/yeast/GSE226769/normalized/train_source_2.csv.gz', compression='gzip', index_col=0)
train_target_4_2 = pd.read_csv('../data/yeast/GSE226769/normalized/train_target_2.csv.gz', compression='gzip', index_col=0)
test_source_4_2 = pd.read_csv('../data/yeast/GSE226769/normalized/test_source_2.csv.gz', compression='gzip', index_col=0)
test_target_4_2 = pd.read_csv('../data/yeast/GSE226769/normalized/test_target_2.csv.gz', compression='gzip', index_col=0)

train_source_4_3 = pd.read_csv('../data/yeast/GSE226769/normalized/train_source_3.csv.gz', compression='gzip', index_col=0)
train_target_4_3 = pd.read_csv('../data/yeast/GSE226769/normalized/train_target_3.csv.gz', compression='gzip', index_col=0)
test_source_4_3 = pd.read_csv('../data/yeast/GSE226769/normalized/test_source_3.csv.gz', compression='gzip', index_col=0)
test_target_4_3 = pd.read_csv('../data/yeast/GSE226769/normalized/test_target_3.csv.gz', compression='gzip', index_col=0)

train_source_4_4 = pd.read_csv('../data/yeast/GSE226769/normalized/train_source_4.csv.gz', compression='gzip', index_col=0)
train_target_4_4 = pd.read_csv('../data/yeast/GSE226769/normalized/train_target_4.csv.gz', compression='gzip', index_col=0)
test_source_4_4 = pd.read_csv('../data/yeast/GSE226769/normalized/test_source_4.csv.gz', compression='gzip', index_col=0)
test_target_4_4 = pd.read_csv('../data/yeast/GSE226769/normalized/test_target_4.csv.gz', compression='gzip', index_col=0)


In [4]:
train_source = pd.concat([train_source_df_1, train_source_df_2, train_source_df_3, train_source_4_1, train_source_4_2, train_source_4_3, train_source_4_4], axis=1)
train_target = pd.concat([train_target_df_1, train_target_df_2, train_target_df_3, train_target_4_1, train_target_4_2, train_target_4_3, train_target_4_4], axis=1)
test_source = pd.concat([test_source_df_1, test_source_df_2, test_source_df_3, test_source_4_1, test_source_4_2, test_source_4_3, test_source_4_4], axis=1)
test_target = pd.concat([test_target_df_1, test_target_df_2, test_target_df_3, test_target_4_1, test_target_4_2, test_target_4_3, test_target_4_4], axis=1)

common_genes = list(train_source.index)

In [5]:
# get network data, training features
tf_set = set()
tf_list_df = pd.read_csv('../data/yeast/yeast_tf_list.tsv.gz', sep='\t', compression='gzip', index_col=0)
for name in tf_list_df['Gene Names']:
    name_splits = name.split(' ')
    for i in name_splits:
        if i.upper() in train_source.index:
            tf_set.add(i.upper())

network_df = pd.read_csv('../data/yeast/yeat_network.csv', index_col=0)
target_gene_list = []
for i, row in network_df.iterrows():
    tf_list = row.tf_list
    if pd.isnull(tf_list): 
        continue
    tf_list = tf_list.split('; ')
    tf_set = tf_set.union(set(tf_list))
    target_gene_list.append(i)

print('Number of TFs used:')
print(len(tf_set))

Number of TFs used:
495


In [6]:
# # get network data, training features
# network_df = pd.read_csv('../data/yeast/yeat_network.csv', index_col=0)
# tf_set = set()
# target_gene_list = []
# for i, row in network_df.iterrows():
#     tf_list = row.tf_list
#     if pd.isnull(tf_list): 
#         continue
#     tf_list = tf_list.split('; ')
#     tf_set = tf_set.union(set(tf_list))
#     target_gene_list.append(i)

# print(len(tf_set))


In [7]:

target_exp = pd.concat([train_target, test_target], axis=1)
source_exp = pd.concat([train_source, test_source], axis=1)
target_gene_list = list(set(target_gene_list).intersection(set(common_genes)))
# filterout NaN target:
target_gene_list = list(target_exp.loc[target_gene_list][target_exp.loc[target_gene_list].isnull().any(axis=1)==False].index)
tf_list = list(tf_set.intersection(set(common_genes)))

res_tf_file = Path("../output/network_model/yeast_tf.csv")
if res_tf_file.is_file():
    tf_list_df = pd.read_csv('../output/network_model/yeast_tf.csv', names=['tf'], index_col=0)
    tf_list = list(tf_list_df.index)

X = source_exp.loc[tf_list]


In [8]:
# GS stats
edge_count = 0
gs_tf_set = set()
gs_target_set = set()
for i, row in network_df.iterrows():
    if i in target_gene_list:
        cur_tf_list = row.tf_list
        gs_target_set.add(i)
        if pd.isnull(cur_tf_list): 
            continue
        cur_tf_list = cur_tf_list.split('; ')
        for cur_tf in cur_tf_list:
            if cur_tf in tf_list:
                gs_tf_set.add(cur_tf)
                edge_count += 1
print('GS edge count:')
print(edge_count)
print('Number of TFs in GS:')
print(len(gs_tf_set))
print('Number of target genes in GS:')
print(len(gs_target_set))




GS edge count:
162100
Number of TFs in GS:
205
Number of target genes in GS:
4794


In [9]:
# filter for high variance targets

new_test_target = test_target.loc[target_gene_list]
new_test_target = new_test_target.loc[new_test_target.std(axis=1) > 0.5]
target_gene_list = new_test_target.index

In [10]:
# setting up MP for calculations
mp_calc = mp_run.MpCalc(target_gene_list, X, network_df, train_source.loc[tf_list], train_target, test_source.loc[tf_list], test_target)
pd.DataFrame(index=mp_calc.tf_list).to_csv('../output/network_model/yeast_tf.csv', header=False)

In [11]:
# Doing all calculations 
mp_calc.full_comp_runner(target_gene_list, '../output/network_model/yeast_full_results.csv.gz', skip_comp=True)

Comparing different regression approaches... ...
Step 1 of 3:


100%|██████████| 385/385 [01:14<00:00,  5.19it/s]

Step 2 of 3:



100%|██████████| 385/385 [00:09<00:00, 41.94it/s]


Step 3 of 3:


100%|██████████| 385/385 [00:08<00:00, 44.24it/s]


Finished comparing all approaches, time elapsed: 92.55277490615845 seconds
Calculating minimal set and disjoint sets... ...
Step 1 of 1:


100%|██████████| 385/385 [06:17<00:00,  1.02it/s]

Finished calculating minimal set and disjoint sets, time elapsed: 378.1896822452545 seconds
